## Merge-ATO-Outputs.py

This script demonstrates how to merge Access to Opportunities (ATO) tables from multiple years into one table. It also formats the columns to match the schema of the table at:

https://data.wfrc.org/datasets/access-to-opportunities-work-related-taz-based?geometry=-117.122%2C40.025%2C-106.669%2C41.481

#### Import required libraries

In [ ]:
import pandas as pd
from simpledbf import Dbf5
import os

#### Build paths to .dbf tables

In [ ]:
# Store highest common directory path
scenarios = os.path.join(os.getcwd(), r'TDM\0 - ModelDev\Official Release\v8.3.1\WF TDM v8.3.1 - 2020-05-08\Scenarios')

# Store path to 2019
path_2019 = os.path.join(scenarios, r'BY_2019\7_PostProcessing\Access_to_Opportunity_2019.dbf')
print(path_2019 + '\n')

# Store path to 2030
path_2030 = os.path.join(scenarios, r'Need_2030\7_PostProcessing\Access_to_Opportunity_2030.dbf')
print(path_2030 + '\n')

# Store path to 2040
path_2040 = os.path.join(scenarios, r'Need_2040\7_PostProcessing\Access_to_Opportunity_2040.dbf')
print(path_2040 + '\n')

# Store path to 2050
path_2050 = os.path.join(scenarios, r'Need_2050\7_PostProcessing\Access_to_Opportunity_2050.dbf')
print(path_2050 + '\n')

#### Load dbf tables into pandas dataframes

In [ ]:
# Read in 2019 as dbf5 object
dbf_2019 = Dbf5(path_2019)

# Read 2019 dbf5 as pandas dataframe object
df_2019 = dbf_2019.to_dataframe()

# 2030
dbf_2030 = Dbf5(path_2030)
df_2030 = dbf_2030.to_dataframe()

# 2040
dbf_2040 = Dbf5(path_2040)
df_2040 = dbf_2040.to_dataframe()

# 2050
dbf_2050 = Dbf5(path_2050)
df_2050 = dbf_2050.to_dataframe()

#### Preview tables (Optional)

In [ ]:
# Show the first 5 rows of a table
print(df_2019.head())
print()

# Show number of rows and columns
print(df_2019.shape)
print()

# Show column names
print(list(df_2019.columns))
print()
print(list(df_2030.columns))
print()

#### Prepping and Formatting tables 

In [ ]:
# Create the base table
base_table = df_2019[['TAZID', 'CO_TAZID', 'DEVACRES']].copy()

# Desired columns to subset by 
columns = ['CO_TAZID', 'HH', 'JOB', 'AUTO_JB', 'AUTO_HH', 'TRAN_JB', 'TRAN_HH', \
           'COMP_AUTO', 'COMP_TRAN']

# subset tables to desired columns
df_2019_subset = df_2019[columns].copy()
df_2030_subset = df_2030[columns].copy()
df_2040_subset = df_2040[columns].copy()
df_2050_subset = df_2050[columns].copy()

# Show Column names before conversion
print("ATO 2040 Column names (BEFORE):")
print(list(df_2040_subset.columns))
print()

# Rename columns, concatenating year to the end
df_2019_subset.columns = ['CO_TAZID', 'HH_19', 'JOB_19', 'JOBAUTO_19', 'HHAUTO_19', 'JOBTRANSIT_19', 'HHTRANSIT_19', \
           'COMPAUTO_19', 'COMPTRANSIT_19']

df_2030_subset.columns = ['CO_TAZID', 'HH_30', 'JOB_30', 'JOBAUTO_30', 'HHAUTO_30', 'JOBTRANSIT_30', 'HHTRANSIT_30', \
           'COMPAUTO_30', 'COMPTRANSIT_30']

df_2040_subset.columns = ['CO_TAZID', 'HH_40', 'JOB_40', 'JOBAUTO_40', 'HHAUTO_40', 'JOBTRANSIT_40', 'HHTRANSIT_40', \
           'COMPAUTO_40', 'COMPTRANSIT_40']

df_2050_subset.columns = ['CO_TAZID', 'HH_50', 'JOB_50', 'JOBAUTO_50', 'HHAUTO_50', 'JOBTRANSIT_50', 'HHTRANSIT_50', \
           'COMPAUTO_50', 'COMPTRANSIT_50']

# Show Column names after conversion
print("ATO 2040 Column names (AFTER):")
print(list(df_2040_subset.columns))
print()

#### Join the tables (Method #1)

In [ ]:
# Join tables to the base table using CO_TAZID field
ato_table = base_table

ato_table = ato_table.merge(df_2019_subset, left_on = 'CO_TAZID', right_on = 'CO_TAZID' , how = 'inner')
ato_table = ato_table.merge(df_2030_subset, left_on = 'CO_TAZID', right_on = 'CO_TAZID' , how = 'inner')
ato_table = ato_table.merge(df_2040_subset, left_on = 'CO_TAZID', right_on = 'CO_TAZID' , how = 'inner')
ato_table = ato_table.merge(df_2050_subset, left_on = 'CO_TAZID', right_on = 'CO_TAZID' , how = 'inner')

#### Join the tables (Method #2)

In [ ]:
# store ATO by year into list
tables = [df_2019_subset, df_2030_subset, df_2040_subset, df_2050_subset]

# Use loop to join each year table to the base table using CO_TAZID field
ato_table = base_table
for table in tables:
    ato_table = ato_table.merge(table, left_on = 'CO_TAZID', right_on = 'CO_TAZID' , how = 'inner')

# Show first 5 rows of merged table
print(ato_table.head())
print()

print(ato_table.shape)
print()

#### Exporting 

In [ ]:
# Store output folder path. There is a hidden .gitignore file here so files written won't be pushed to github
temp = os.path.join(os.getcwd(), 'Results')

# Create name for output csv
out_table = os.path.join(temp, 'ATO.csv')

# export data frame to csv
ato_table.to_csv(out_table, index=False)

#### The final step would be joining the output table to a TAZ shapefile/feature dataset using ArcGIS.